## Startup Process
```mermaid
flowchart LR
    C[Serial.begin] --> D[strip.begin]
    D --> E[updateLength/setBrightness]
    E --> H[init stripData n oldStripData]
    H --> I[initializeBLE_fn]
```


> **Description:**
>
> The startup process initializes the LED strip controller by setting up serial communication, configuring the LED strip parameters (length, brightness, color order), and allocating memory for the strip data structures. It then instantiates the Bluetooth process by calling `initializeBLE`, which prepares the device for wireless communication. Only after these steps are complete does the main loop begin, allowing the system to respond to incoming messages and update the LED strip in real time. This ensures that all hardware and communication interfaces are ready before any data processing or animation occurs.

## Bluetooth Process
```mermaid
flowchart TD
    J[onWrite event]
    J --> K[parseAndUpdateData_fn]
    K --> L[myData.updated = true]
```


> **Description:**
>
> The `parseAndUpdateData` function is called when a Bluetooth message is received. It takes a JSON string and parses it into the `myData` structure, updating fields such as brightness, colors, mode, and other parameters. If parsing is successful, it sets `myData.updated = true`, signaling that new data has arrived and the LED strip should be refreshed with the new settings. This flag ensures that the main loop processes the new configuration and applies changes to the hardware.

## Main Loop Process
```mermaid
flowchart TD
    A[loop_fn] --> B[handleStrip_fn]
    B -- 1.mydata.updated --> E[cloneData_fn, setBrightness, length, type] 
    E --> H[strip.clear then show]
    H --> J[transitionValue = 100]
    B -- 2.update.stripData --> K[callModeFunction_fn]
    B -- 3.display.stripData --> L{transitionValue < 2?}
    L -- Yes --> M[show_fn]
    L -- No --> N[blendAndShow_fn]
    M --> O[strip.setPixelColor]
    O --> P[strip.show]
    N --> Q[blendColors_fn]
    Q --> R[strip.setPixelColor]
    R --> S[strip.show]
```

> **Description:**
>
> The `handleStrip` function is the core routine that manages updates to the LED strip. When `myData.updated` is true, it refreshes the strip settings, copies the previous data for transitions, and prepares the strip for new effects. It then calls `callModeFunction`, which applies the selected lighting mode based on the current configuration. The strip is updated either directly or through blending, depending on the value of `transitionValue`. If a transition is in progress, colors are smoothly blended between the old and new states. The `updated` parameter acts as a trigger, ensuring that changes are only processed when new data arrives, making the system responsive to Bluetooth or other input events.

# Lighting Basics

- Static: Passive, no animation.
- Dynamic: Active, animates over time.
- Modifier: Alters or combines effects; can be static or dynamic, and can be layered.

# myData Attributes

| Attribute      | Type      | Description                                               |
|---------------|-----------|-----------------------------------------------------------|
| brightness    | int       | LED strip brightness (0-100)                              |
| lightMode     | char[32]  | Current lighting mode (e.g., "static", "shift")          |
| colorOne      | uint32_t  | Primary color (RGB, 0xRRGGBB)                             |
| colorTwo      | uint32_t  | Secondary color (RGB, 0xRRGGBB)                           |
| colorThree    | uint32_t  | Tertiary color (RGB, 0xRRGGBB)                            |
| ledCount      | int       | Number of LEDs in the strip                               |
| maxCurrent    | int       | Maximum allowed current (mA)                              |
| colorOrder    | uint16_t  | Color order for the strip (e.g., NEO_GRB + NEO_KHZ800)    |
| pixelPin      | int       | GPIO pin connected to the LED strip                       |
| pixelCount    | int       | Number of pixels (usually same as ledCount)               |
| speed         | int       | Animation speed (0-100)                                   |
| intensity     | int       | Effect intensity (0-100)                                  |
| direction     | int       | Animation direction (0=forward, 1=reverse, 2=bounce)      |
| count         | int       | Count parameter for effects (e.g., number of segments)     |
| updated       | bool      | True if new data has arrived and should be processed       |

# Effects and Modes

Color coding:

- Light green: Static modes
- Light orange: Static modifier modes 
- Light purple: Dynamic modifier modes 
- Light blue: Dynamic modes


```mermaid
graph LR

  %% Static (light green, dark text)
  mode_static -- colorOne --> effect_static
  mode_static_tri -- colorOne, colorTwo, colorThree --> effect_static

  %% Modifiers (static and dynamic)
  mode_percent -- intensity, speed, colorOne --> effect_range
  mode_percent_tri -- intensity, speed, colorOne, colorTwo, colorThree --> effect_range
  mode_shift -- direction, inherits colors from previous mode --> effect_shift
  mode_washing_machine -- direction, count, intensity, colorOne, colorTwo --> effect_shift
  mode_breath -- speed, intensity, colorOne, colorTwo, colorThree, inherits colors from previous mode --> effect_fade
  mode_candle -- colorOne, count, intensity, speed --> custom_candle_fn
  mode_blink -- speed, colorOne --> effect_blink
  mode_blink_toggle -- speed, colorOne, colorTwo --> effect_blink
  mode_blink_random -- speed --> effect_blink
  mode_blink_random -- randomColor --> randomColor

  %% Effects used by Modes (dynamic listed last)
  mode_swipe_random -- direction --> randomColor
  mode_swipe_random -- direction --> effect_swipe
  mode_swipe -- direction, colorOne, colorTwo --> effect_swipe 
  mode_colorloop -- speed --> Wheel
  mode_sweep -- direction, colorOne, intensity, count --> effect_sweep
  mode_sweep_dual -- direction, colorOne, colorTwo, intensity, count --> effect_sweep
  mode_pacifica -- custom params --> custom_layered_waves
  mode_sunrise -- custom params --> custom_gradient_blend
  mode_aurora -- custom params --> custom_palette_blend

  %% Color coding:

  %% Static Modifier (light orange, dark text)
  style mode_percent fill:#ffd580,color:#222
  style mode_percent_tri fill:#ffd580,color:#222

  %% Dynamic Modifier (light purple, dark text)
  style mode_shift fill:#d1b3ff,color:#222
  style mode_washing_machine fill:#d1b3ff,color:#222
  style mode_breath fill:#d1b3ff,color:#222
  style mode_blink fill:#d1b3ff,color:#222
  style mode_blink_toggle fill:#d1b3ff,color:#222
  style mode_blink_random fill:#d1b3ff,color:#222
  style mode_candle fill:#d1b3ff,color:#222

  %% Static (light green, dark text)
  style mode_static fill:#b3e6b3,color:#222
  style mode_static_tri fill:#b3e6b3,color:#222

  %% Dynamic (light blue, dark text)
  style mode_swipe fill:#b3c6e6,color:#222
  style mode_swipe_random fill:#b3c6e6,color:#222
  style mode_colorloop fill:#b3c6e6,color:#222
  style mode_sweep fill:#b3c6e6,color:#222
  style mode_sweep_dual fill:#b3c6e6,color:#222
  style mode_pacifica fill:#b3c6e6,color:#222
  style mode_sunrise fill:#b3c6e6,color:#222
  style mode_aurora fill:#b3c6e6,color:#222
```

  mode_stream --> stub
  mode_palette --> stub
  mode_plasma --> stub
  mode_perlin_move --> stub
  mode_heartbeat --> stub
  mode_twinkles --> stub
  mode_sinelon_dual --> stub
  mode_theater --> stub
  mode_fireworks --> stub
  mode_juggle --> stub
  mode_bouncing_balls --> stub
  mode_meteor --> stub
  mode_tetrix --> stub